<a href="https://colab.research.google.com/github/krmcclelland/Amazon_Vine_Analysis/blob/main/16_6_2_mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Module 16.6.2 Stop Words
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()
# Skill Drill 16.6.2 added to make work
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import Tokenizer

In [ ]:
# Create DataFrame
sentenceData = spark.createDataFrame([
                                      (0, ["Big","data","is","super","powerful"]),
                                      (1,["This","is","going","to","be","epic"])
],["id","raw"])

In [ ]:
# Import stop words library
from pyspark.ml.feature import StopWordsRemover

In [ ]:
# Run the Remover
remover = StopWordsRemover(inputCol="raw", outputCol="filtered")

In [ ]:
#Tansform and show data
remover.transform(sentenceData).show(truncate=False)

+---+--------------------------------+----------------------------+
|id |raw                             |filtered                    |
+---+--------------------------------+----------------------------+
|0  |[Big, data, is, super, powerful]|[Big, data, super, powerful]|
|1  |[This, is, going, to, be, epic] |[going, epic]               |
+---+--------------------------------+----------------------------+



In [ ]:
# Skill Drill 16.6.2 Combine a tokenizer and a stopword remover on a datafram not broken out
# Create sample Dataframe
dataframe_mod = spark.createDataFrame([
                                   (0, "Spark is great"),
                                   (1, "We are learning Spark"),
                                   (2, "Spark is better than hadoop no doubt"),
                                   (3, "Adding a line for skill drill practice")
],["id","sentence"])
dataframe_mod.show(truncate=False)

+---+--------------------------------------+
|id |sentence                              |
+---+--------------------------------------+
|0  |Spark is great                        |
|1  |We are learning Spark                 |
|2  |Spark is better than hadoop no doubt  |
|3  |Adding a line for skill drill practice|
+---+--------------------------------------+



In [ ]:
# Skill Drill 16.6.2 Combine a tokenizer and a stopword remover on a datafram not broken out
# Create a function to return the length of a list
def word_list_length(word_list):
    return len(word_list)
# Create a user defined function   
count_tokens = udf(word_list_length, IntegerType())

In [ ]:
# Skill Drill 16.6.2 Combine a tokenizer and a stopword remover on a datafram not broken out
# Create our Tokenizer
tokenizer = Tokenizer(inputCol="sentence",outputCol="words")

# Transform and show DataFrame
dataframe_mod = tokenizer.transform(dataframe_mod)

# Select the needed columns and don't truncate results
dataframe_mod.withColumn("tokens",count_tokens(col("words"))).show(truncate=False)

+---+--------------------------------------+----------------------------------------------+------+
|id |sentence                              |words                                         |tokens|
+---+--------------------------------------+----------------------------------------------+------+
|0  |Spark is great                        |[spark, is, great]                            |3     |
|1  |We are learning Spark                 |[we, are, learning, spark]                    |4     |
|2  |Spark is better than hadoop no doubt  |[spark, is, better, than, hadoop, no, doubt]  |7     |
|3  |Adding a line for skill drill practice|[adding, a, line, for, skill, drill, practice]|7     |
+---+--------------------------------------+----------------------------------------------+------+



In [ ]:
# Skill Drill 16.6.2 Combine a tokenizer and a stopword remover on a datafram not broken out
# Run the Remover
remover_mod = StopWordsRemover(inputCol="words", outputCol="filtered")

In [ ]:
# Skill Drill 16.6.2 Combine a tokenizer and a stopword remover on a datafram not broken out
#Tansform and show data
remover_mod.transform(dataframe_mod).show(truncate=False)

+---+--------------------------------------+----------------------------------------------+--------------------------------------+
|id |sentence                              |words                                         |filtered                              |
+---+--------------------------------------+----------------------------------------------+--------------------------------------+
|0  |Spark is great                        |[spark, is, great]                            |[spark, great]                        |
|1  |We are learning Spark                 |[we, are, learning, spark]                    |[learning, spark]                     |
|2  |Spark is better than hadoop no doubt  |[spark, is, better, than, hadoop, no, doubt]  |[spark, better, hadoop, doubt]        |
|3  |Adding a line for skill drill practice|[adding, a, line, for, skill, drill, practice]|[adding, line, skill, drill, practice]|
+---+--------------------------------------+---------------------------------------